In [ ]:
# Download model (replace with actual link)
!wget '' -O Name.safetensors

In [ ]:
# Install dependencies
!apt-get -y install git
!pip install pygit2==1.15.1

# Clone Fooocus
!git clone https://github.com/dhaneshsaini/Fooocus.git /content/Fooocus
%cd /content/Fooocus

# Run Fooocus
!python entry_with_update.py --share --always-high-vram --preset pixel